<a href="https://colab.research.google.com/github/igorcsouza/formula-1/blob/main/Apostas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Para usar expressões regulares
import re
# Para dicionários que inicializam automaticamente listas vazias
from collections import defaultdict

# Cole aqui as mensagens copiadas do WhatsApp
mensagens = """
[30/7 10:00] Fabiano Anacleto: PIA
PIA
NOR
LEC
PIA
[30/7 10:01] Mateus Campolina: NOR
PIA
NOR
VER
PIA
[30/7 10:05] Humberto Santos Guarim: PIA
PIA
VER
NOR
VER
[30/7 10:07] Murilo Borba Donald: PIA
PIA
NOR
VER
PIA
[30/7 10:07] Pedro Bordone: PIA
PIA
VER
NOR
VER
[30/7 10:31] Doug M. Martinho: NOR
NOR
PIA
LEC
NOR
[30/7 10:42] Leonardo Palmerston: PIA
PIA
NOR
VER
LEC
[30/7 10:46] Alexandre Cuco: PIA
PIA
NOR
RUS
PIA
[30/7 10:52] Igor Souza: NOR
NOR
PIA
LEC
RUS
[30/7 11:02] Bruno Pavão: NOR
NOR
PIA
VER
NOR
[30/7 11:08] Igor Novaes: Pia
Pia
Nor
Ver
Ham
[30/7 11:33] Erico Batche: Ver
Ver
Nor
Pia
Pia
[30/7 12:01] Antonio Pellegrini: PIA
PIA
VER
NOR
PIA
[30/7 13:03] João Felipe Verleun: NOR
VER
NOR
PIA
VER
[31/7 11:19] Vitor Lousada: NOR
NOR
PIA
VER
NOR
[1/8 10:25] Marcelo Sadock: PIA
PIA
NOR
LEC
NOR
"""  # Cole as mensagens acima

# Extrair o sobrenome (ignorando preposições como "de", "da", etc.)
def extrair_sobrenome(nome_completo):
    partes = nome_completo.strip().split()
    for parte in reversed(partes):
        if parte.lower() not in {"da", "de", "do", "dos", "das"}:
            return parte.upper()
    return partes[-1].upper()

# Função principal que processa o bloco de mensagens
def processar_mensagens(texto):
    # Regex para identificar a linha de início de uma mensagem
    padrao = re.compile(r"\[(\d+)/(\d+)\s+(\d+):(\d+)]\s+(.*?):\s*(.*)")

    apostas = []  # Lista que guardará os dados extraídos de cada apostador
    linhas = texto.strip().splitlines()  # Quebra o texto em linhas

    i = 0
    dia_base = None # Dia da primeira aposta
    dias_ajustados = {}  # Mapeia data (dia/mês) para o número de dias somados
    incremento_dia = 0  # Incremento para diferenciar dias consecutivos

    # Percorre todas as linhas da mensagem
    while i < len(linhas):
        # Verifica se a linha atual corresponde ao padrão de data e hora (início de uma nova mensagem)
        if padrao.match(linhas[i]):
            # Se corresponder, extrai as informações usando o padrão regex
            match = padrao.match(linhas[i])

            # Extrai dia, mês, hora, minuto e nome do apostador
            dia = int(match.group(1))
            mes = int(match.group(2))
            hora = int(match.group(3))
            minuto = int(match.group(4))
            nome = match.group(5).strip()

            # Ajusta o cálculo das horas levando em conta a virada de dia.
            # Cada novo dia detectado é tratado como +24h em relação ao anterior,
            # garantindo que o horário de envio continue crescente ao longo dos dias.
            # Por exemplo, se alguém aposta às 00:10 no dia seguinte, isso será tratado como 24:10.

            # Cria uma "chave de data" no formato DD/MM (com zero à esquerda se necessário)
            # para identificar de forma única cada dia nas mensagens e ajudar a ajustar o horário
            # total em horas ao longo de múltiplos dias.
            data_chave = f"{dia:02d}/{mes:02d}"
            if dia_base is None:
                # Armazena o primeiro dia como base e define incremento 0 para ele
                dia_base = data_chave
                dias_ajustados[data_chave] = 0
            elif data_chave not in dias_ajustados:
                # Para cada novo dia encontrado, incrementa +1 (equivalente a +24h)
                incremento_dia += 1
                dias_ajustados[data_chave] = incremento_dia

            # Converte a hora para o total de horas considerando os dias somados
            horas_totais = hora + dias_ajustados[data_chave] * 24

            # Lê os 5 palpites das próximas linhas e coloca tudo em maiúsculas
            palpites = [linhas[i + j].strip().upper() for j in range(1, 6)]

            # Adiciona o registro de aposta
            apostas.append({
                "hora": horas_totais,
                "minuto": minuto,
                "nome": nome,
                "sobrenome": extrair_sobrenome(nome),
                "palpites": palpites
            })

            # Avança 6 linhas: 1 com horário + nome + 5 com palpites
            i += 6
        else:
            i += 1

    # Agrupa todas as apostas que foram feitas no mesmo minuto
    # Isso é importante porque, em caso de empate na pontuação,
    # o critério de desempate é a ordem de envio da aposta.
    # Como o WhatsApp não fornece segundos, precisamos criar subgrupos
    # dentro de cada minuto para depois ordenar pela ordem em que foram recebidas.
    grupos = defaultdict(list)
    for aposta in apostas:
        chave = (aposta["hora"], aposta["minuto"]) # chave representa o minuto completo (ex: 10:42)
        grupos[chave].append(aposta) # adiciona a aposta na lista correspondente a esse minuto

    saida = []

    # Constrói as linhas com código do apostador, horário e palpites
    for (hora, minuto), grupo in grupos.items():
        for idx, aposta in enumerate(grupo):
            segundo = idx  # Segundo simula a ordem dentro do mesmo minuto
            horario_formatado = f"{hora:02d}:{minuto:02d}:{segundo:02d}"  # Formato HH:MM:SS
            cabecalho = aposta["sobrenome"][:3].upper()  # Usa as 3 primeiras letras do sobrenome
            linha = [cabecalho, horario_formatado] + aposta["palpites"]
            saida.append(linha)

    # Ordena as linhas pelo código do sobrenome
    saida_ordenada = sorted(saida, key=lambda x: x[0])

    # Transpõe linhas para colunas (cada coluna é de um apostador)
    colunas = list(zip(*saida_ordenada))

    # Junta os dados usando ";" como separador para colar em planilha Google
    texto_final = "\n".join(";".join(coluna) for coluna in colunas)
    return texto_final

# Pré-processamento: quebra linha logo após os dois pontos
# Isso garante que o palpite da pole fique em uma linha separada
texto = re.sub(r"(\[\d+/\d+ \d+:\d+\] .+?:)", r"\1\n", mensagens)

# Executa o processamento e exibe resultado
print(processar_mensagens(texto))

ANA;BAT;BOR;CAM;CUC;DON;GUA;LOU;MAR;NOV;PAL;PAV;PEL;SAD;SOU;VER
10:00:00;11:33:00;10:07:01;10:01:00;10:46:00;10:07:00;10:05:00;35:19:00;10:31:00;11:08:00;10:42:00;11:02:00;12:01:00;58:25:00;10:52:00;13:03:00
PIA;VER;PIA;NOR;PIA;PIA;PIA;NOR;NOR;PIA;PIA;NOR;PIA;PIA;NOR;NOR
PIA;VER;PIA;PIA;PIA;PIA;PIA;NOR;NOR;PIA;PIA;NOR;PIA;PIA;NOR;VER
NOR;NOR;VER;NOR;NOR;NOR;VER;PIA;PIA;NOR;NOR;PIA;VER;NOR;PIA;NOR
LEC;PIA;NOR;VER;RUS;VER;NOR;VER;LEC;VER;VER;VER;NOR;LEC;LEC;PIA
PIA;PIA;VER;PIA;PIA;PIA;VER;NOR;NOR;HAM;LEC;NOR;PIA;NOR;RUS;VER
